# Download image

In [21]:
import geopandas as gpd
from tqdm import tqdm
import pandas as pd
import os

In [3]:
# bulding_shp_file = r'https://github.com/gladcolor/TGI_building/raw/master/flood_zone_footprint.zip'
bulding_shp_file = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\TGI\Vector\building_footprint_boroughs.shp'

buildings_gdf = gpd.read_file(bulding_shp_file).to_crs(epsg=4326)
print("Row count:", len(buildings_gdf))
buildings_gdf.head()

Row count: 1084221


,Join_Count,TARGET_FID,BIN,boro_name,Shape_Leng,Shape_Area,geometry
0,1,0,3170958,Brooklyn,185.034390,1638.139229,"POLYGON ((-73.96665 40.62600, -73.96685 40.625..."
1,1,1,5028452,Staten Island,109.884570,753.604121,"POLYGON ((-74.16790 40.63936, -74.16790 40.639..."
2,1,2,5078368,Staten Island,109.433052,718.949579,"POLYGON ((-74.19511 40.55611, -74.19520 40.556..."
3,1,3,3245111,Brooklyn,183.800502,1167.664121,"POLYGON ((-73.96113 40.57744, -73.96115 40.577..."
4,1,4,4161096,Queens,180.764206,1754.709217,"POLYGON ((-73.75422 40.75591, -73.75417 40.755..."


In [13]:
boroughs = pd.DataFrame(buildings_gdf['boro_name'].value_counts()).index.to_list()[::-1]
boroughs

['Manhattan', 'Bronx', 'Staten Island', 'Brooklyn', 'Queens']

In [ ]:
import requests
from PIL import Image
import io

def get_image(row, naming_column, saved_path, size=512,  extend_factor=1.3):
    
    try:
        # Calculate bounding box from geometry
        bbox = row.geometry.bounds
        minx, miny, maxx, maxy = bbox

            # Calculate width and height
        width = maxx - minx
        height = maxy - miny

        # Extend the bounding box
        minx = minx - width * (extend_factor - 1) / 2
        miny = miny - height * (extend_factor - 1) / 2
        maxx = maxx + width * (extend_factor - 1) / 2
        maxy = maxy + height * (extend_factor - 1) / 2

        bbox_str = f"{minx},{miny},{maxx},{maxy}"
        # print(bbox_str)
    # 
        # Compose the WMS URL
        url = f"https://orthos.its.ny.gov/arcgis/rest/services/wms/Latest/MapServer/export?bbox={bbox_str}&bboxSR=4269&layers=show:0,1,2&format=png&size={size},{size}&imageSR=3857&f=image"
        # print(url)

        # Send a GET request to the URL
        response = requests.get(url, stream=True)

        # Ensure the request was successful
        response.raise_for_status()

        # Open the image
        img = Image.open(io.BytesIO(response.content))

        # Save the image
        filename = f"{saved_path}/{row[naming_column]}.png"
        img.save(filename)
        
    except Exception as e:
        print("Error:", e)
        # continue
    

# saved_path = r'P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone'
saved_path = r'D:\Research\TGI\Raster\building_roof_images'



naming_column = 'BIN'   
tqdm.pandas(desc="Downloading images")

for borough in boroughs:
    borough_dir = os.path.join(saved_path, borough)
    os.makedirs(borough_dir, exist_ok=True)
    gdf = buildings_gdf.query(f" boro_name == '{borough}' ")
    borough_building_cnt = len(gdf)
    print(borough_building_cnt)

    gdf.iloc[:].progress_apply(get_image, naming_column=naming_column, saved_path=borough_dir, axis=1)

45428


In [22]:
url = "https://orthos.its.ny.gov/arcgis/rest/services/wms/Latest/MapServer/export?bbox=-74.26,40.49,-73.70,40.92&bboxSR=4326&layers=show:0,1,2&format=png&size=1200,800&imageSR=4326&f=image"


time: 16 ms (started: 2023-07-07 11:07:42 -04:00)


In [36]:
import brails

ModuleNotFoundError: No module named 'brails'

In [9]:
# ! pip install BRAILS
# ! pip install nodejs

# !pip install jupyterlab_execute_time

# Detect

In [2]:
import brails
from brails.modules import PytorchRoofClassifier, PytorchOccupancyClassifier


from glob import glob
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)  # Show full content of each column

roofModel = PytorchRoofClassifier()


Default roof type classifier will be used: transformer_rooftype_v1.

Determining the roof type for each building...
Default roof type classifier model at tmp/models loaded

Model found locally: ./tmp/models/transformer_rooftype_v1.pkl 
Loading  ./tmp/models/transformer_rooftype_v1.pkl


In [4]:
img_dir = r'P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone'
imgs = glob(os.path.join(img_dir, "*.png"))
len(imgs)


48283

In [11]:
# for i in range(0, len(imgs), 100):
#     stop_pos = 

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

  Using cached ipython_autotime-0.3.1-py2.py3-none-any.whl (6.8 kB)
time: 0 ns (started: 2023-07-07 11:05:23 -04:00)


In [12]:
predictions = roofModel.predictMultipleImages(imgs[:100])
predictions = predictions.replace({"flat": "gabled", 'gabled':'hipped', 'hipped':'flat'})
# predictions.to_csv(r'P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\roof_type_prediction.csv', index=False)

                                                                               image  \
0   P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_5053676.png   
1   P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_5052970.png   
2   P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_4299828.png   
3   P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_3227038.png   
4   P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_4590943.png   
..                                                                               ...   
95  P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_5134914.png   
96  P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_5164152.png   
97  P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_5127569.png   
98  P:\Shared drives\T5\Research\TGI\footprint_image_flood_zone\building_5135425.png   
99  P:\Shared drives\T5\Research